<a href="https://colab.research.google.com/github/vlad98rus/-netology_pyda/blob/HomeWork-1/DZ_linear_method_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к занятию "Алгоритмы классификации: линейные методы, логистическая регрессия и SVM"

Условие: Задача состоит в модификации кода градиентного спуска для задачи регрессии. Необходимо, чтобы ваш градиентный спуск мог решать задачи классификации, а именно линейной классификации и логистической регрессии.
На всякий случай напомню, что код будет разный, поэтому жду от вас две функции градиентного спуска.
Чтобы решить задачи, обратите внимание на пункт “Принцип максимального правдоподобия” статьи http://www.machinelearning.ru/wiki/images/6/68/voron-ML-Lin.pdf . Там находится ответ на вопрос “какая функция потерь”.
Все обсуждения ведутся в треде этого поста.
НИКАКИХ ГОТОВЫХ РЕШЕНИЙ ДАВАТЬ НЕЛЬЗЯ. Здесь только обсуждается путь к решению задачи.
Успеха!

In [0]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

Отключаем предупреждения

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
RANDOM_STATE = 999
N_FOLDS = 5

In [0]:
raw_df = pd.read_csv('adult.csv')

In [43]:
raw_df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [44]:
raw_df.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


Разделяем наши колонки

In [45]:
num_cols = ['age','fnlwgt','educational-num','capital-gain','capital-loss','hours-per-week']
cat_cols = raw_df.drop(num_cols+['income'], axis=1).columns
print(cat_cols)

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'gender', 'native-country'],
      dtype='object')


In [0]:
df = pd.get_dummies(raw_df, columns = cat_cols)

In [47]:
df.columns

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'income', 'workclass_?', 'workclass_Federal-gov',
       'workclass_Local-gov',
       ...
       'native-country_Portugal', 'native-country_Puerto-Rico',
       'native-country_Scotland', 'native-country_South',
       'native-country_Taiwan', 'native-country_Thailand',
       'native-country_Trinadad&Tobago', 'native-country_United-States',
       'native-country_Vietnam', 'native-country_Yugoslavia'],
      dtype='object', length=109)

In [48]:
df['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [0]:
df['income'] = df['income'].apply(lambda x: 0 if x == '<=50K' else 1)

In [50]:
X, Y = df.drop(['income'], axis=1), df['income']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = RANDOM_STATE)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(34189, 108) (14653, 108) (34189,) (14653,)


In [0]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
gridsearch = GridSearchCV(LogisticRegression(n_jobs=-1,random_state=RANDOM_STATE), {'penalty': ['l1','l2'], 'C': [0.01, 0.05]}, scoring='neg_log_loss', cv=N_FOLDS)
gridsearch.fit(X_train, y_train)
best_model = gridsearch.best_estimator_
y_pred = best_model.predict(X_test)

In [53]:
print(f"LogisticRegression score={best_model.score(X_test, y_test)}")

LogisticRegression score=0.8493823790350099


In [54]:
print (classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90     11185
           1       0.72      0.59      0.65      3468

    accuracy                           0.85     14653
   macro avg       0.80      0.76      0.78     14653
weighted avg       0.84      0.85      0.84     14653

